In [1]:
import pandas as pd
import pmlb
import altair as alt
import numpy as np
from sklearn.model_selection import train_test_split

## Data Preparation

In [5]:
df = pmlb.fetch_data('pima')

# impute the missing input feature values with the median of the target class  
imputeFeatures = ['plasma glucose', 'Diastolic blood pressure', 'Triceps skin fold thickness', 'Body mass index', '2-Hour serum insulin']
for feature in imputeFeatures:
    df.loc[(df.target==0) & (df[feature] == 0), feature] = df[df.target==0][feature].median()
    df.loc[(df.target==1) & (df[feature] == 0), feature] = df[df.target==1][feature].median()

In [7]:
# split and normalized 
X = df.drop(['target'], axis=1)
Y = df['target']
X_normalized=(X - X.mean()) / X.std()

## Visualize Mean Absolute Value For Each Feature 

In [ ]:
feature_important_weights = X.melt(ignore_index=False)
feature_important_weights.columns = ['feature', 'feature_value']

# shap value for each feature value 
shapley_values = shap_df.melt(ignore_index=False)
feature_important_weights['shap_value'] = shapley_values['value']

# lime weight for each feature value
lime_weights = lime_df.melt(ignore_index=False)
feature_important_weights['lime_weights'] = lime_weights['value'] 

In [ ]:
# Summary Strip Plot